In [3]:
from board import Board
from queue import PriorityQueue
import copy
import numpy as np
import time

def selection(a):
    list = []
    denom = 0
    
    for state in a:
        #print(state.get_fit_inverse())
        denom += state.get_fit_inverse()
    
    #print("denom: " + str(denom))
    
    for state in a:
        list.append(((state.get_fit_inverse() / denom), state))
        #print(state.get_fit_inverse() / denom)
        
    selected = []
    
    for k in range(len(list)):
        base = 0
        r = np.random.rand()
        for i in range(len(list)):
            #print(str(base) + " " + str(r) + " " + str((base + (list[i][0]))))
            
            if base <= r < (base + (list[i][0])):
                #print("state found " + str(base))
                selected.append(copy.deepcopy(list[i][1]))
                break
                
            else:
                #print("failed " + str(base))
                base += (list[i][0])
    
    return selected


def cross_over(a):

    for i in range(0, len(a), 2):
        #get the slice index
        r = np.random.randint(0, 4) + 1
        
        #print(r)
        #print("Pair: \n       " + str(a[i].get_map()) + '\n       ' + str(a[i + 1].get_map()))
        
        map1_slice1 = a[i].get_map()[0:r]
        map1_slice2 = a[i + 1].get_map()[r:5]
        
        map2_slice1 = a[i + 1].get_map()[0:r]
        map2_slice2 = a[i].get_map()[r:5]        
        
        #print("slice1 " + str(map1_slice1) + " - " + str(map1_slice2))
        #print("slice2 " + str(map2_slice1) + " - " + str(map2_slice2))        
   
        combined_map1 = map1_slice1 + map1_slice2
        combined_map2 = map2_slice1 + map2_slice2
        
        #print('map 1: ' + str(map1))
        #print('map 2: ' + str(map2))
        
        a[i].set_map(combined_map1)
        a[i + 1].set_map(combined_map1)
        
        #print('set 1: ' + str(a[i].get_map()))
        #print('set 2: ' + str(a[i + 1].get_map()))

    return a

def mutation(a):
    
    for state in a:
        state_map = state.get_map()
        row_index = np.random.randint(0, len(state_map))
        row_list = state_map[row_index]
        
        #zero the row but get the index that has a 1 so that we do not mutate to the same state
        for i in range(len(row_list)):
            if(row_list[i] == 1):
                clear_index = i
                
        #get the mutation index, make sure its not the same as the initial state        
        mutation_index = np.random.randint(0, len(state_map))
        while(mutation_index == clear_index):
            mutation_index = np.random.randint(0, len(state_map))
        
        #print(str(row_index) + " " + str(mutation_index))
        state.flip(row_index, mutation_index)
        state.flip(row_index, clear_index)
        
    return a

def check_pop(pop):
    fitness_sum = 0
    
    for item in pop:
        item.fitness()
        fitness_sum += item.get_fit()
        
        if(item.get_fit() == 0):
            return True, fitness_sum
        
    return False, fitness_sum
                
     
def genetic_a(pop, threshold, iterations):
    restart_threshold_counter = 0
    #Not sure what this value should be. This is very rarely reached.
    threshold_comparator = pop[0].get_max_fit() * len(pop) *.5
    
    #Not sure what this number should be, this should act like a random restart.
    for i in range(0, iterations):
        sel = selection(pop)
        cross = cross_over(sel)
        pop = mutation(cross)

        found, fitness_sum = check_pop(pop)

        #print("fitness sum:" + str(fitness_sum))
        if(found):
            return True, pop
        
        #this should attempt to detect being stuck at a local minima. Not sure what the thresholds and parameters should be.
        if(fitness_sum >= threshold_comparator):
            restart_threshold_counter += 1
        else:
            restart_threshold_counter = 0
                        
        if(restart_threshold_counter > threshold):
            return False, pop

    return False

if __name__ == '__main__':
    iterations= 500
    restart_threshold = 3
    result_found = False
    start_time = time.time()
    restart_counter = -1;

    while(not result_found):
        restart_counter += 1
        pop = []
        for i in range(8):
            b = Board(5)
            b.fitness()
            pop.append(b)
        result_found, pop = genetic_a(pop, restart_threshold, iterations)

    end_time = int((time.time() - start_time) * 1000)
    print('Running Time:', end_time, 'ms')
    print('# of restarts: ' + str(restart_counter))
    for item in pop:
        if item.get_fit() == 0:
            item.show()

Running Time: 8 ms
# of restarts: 0
[[0 0 0 1 0]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [1 0 0 0 0]]
Fitness:  0
